# Southeast Blueprint 2023 Extent and subregions

This creates the full extent of the 2023 Southeast blueprint.

It uses the 2022 base blueprint extent as an input. It also uses the marine and Caribbean extent as an input.

Created by Amy Keister, last run by Amy Keister 6 July 2023. It tookk 12 minutes to run.

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\2_FullExtent\FullExt.gdb"

In [39]:
# define final indicator outputs
Out1= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprintForseableMaxExtentPossibleIssueWithDeepMarineBoundary.tif"
Out2= "SEBlueprint2023SubregionsVIPR6"
Out2Folder= r"D:\SE_Blueprint_2023\2_FullExtent"
Out3= r"D:\SE_Blueprint_2023\2_FullExtent\SEBlueprint2023ExtentVIPR6.tif"

In [5]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [6]:
# define inputs
baseSub= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintSubRgn.shp"
base= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"
SEextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\SE_Blueprint_2022.tif"
usMar= r"F:\GIS_DATA\Boundaries\USMaritimeLimitsAndBoundariesSHP\USMaritimeLimitsNBoundaries.shp"
VIPR= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"
MarineRaster= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_Extent_v1.tif"
#GulfM= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_GoM_v1.shp"
#AtlanticM= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_SA_v1.shp"
#SAm= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_SA_v1.tif"
GOMm= r"D:\SE_Blueprint_2023\2_MarineExtents\Marine_GoM_v1.tif"
Square= r"D:\BaseBlueprint\2022\1_SubregionsAndExtent\SquareForTesting.gdb\square"

## Start Analysis

In [7]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [8]:
# print workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\2_FullExtent\FullExt.gdb


### Try to estimate bounding largest possible bounding box for SE Blueprint for the forseable future

This should include all SECAS states, Gulf, Atlantic, and Caribbean US maritime extents

In [9]:
# select VIPR from US maritime limits
arcpy.analysis.Select(usMar, "usMarVIPR", "REGION = 'Puerto Rico and U.S. Virgin Islands'")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\usMarVIPR'>

In [10]:
# convert VIPR US martime limits from line to polygon
arcpy.management.FeatureToPolygon("usMarVIPR", "usMarVIPRp", None, "ATTRIBUTES", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\usMarVIPRp'>

In [11]:
# add and calculate field to convert to raster
arcpy.management.CalculateField("usMarVIPRp", "raster", "1", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\usMarVIPRp'>

In [12]:
# combine US maritime limits of VIPR with 2022 base blueprint subregions
# this should get us the maximum extent
arcpy.analysis.Union([baseSub, "usMarVIPRp"], "BaseVIPR", "ONLY_FID", None, "GAPS")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\BaseVIPR'>

In [13]:
# buffer so the extent will be big enough for corridor analysis
# even though this doesn't include the Atantic and Gulf marine extent as inputs, I think it will be big enough to 
# include those areas in the bounding box extent, which is all I want this for
arcpy.analysis.Buffer("BaseVIPR", "BaseVIPR27km", "27 Kilometers", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\BaseVIPR27km'>

In [14]:
# convert to raster and snap to base blueprint 2022 extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="usMarVIPRp", field="raster", out_raster="usMarVIPRr", cell_size=Rextent)

In [36]:
# use cell statistics to combine the Marine raster, base blueprint extent, and estimated max extent of VIPR
# use the buffered combination of Base blueprint subregions and VIPR as the extent, 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="BaseVIPR27km", snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics([MarineRaster, base, "usMarVIPRr"], "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("maxBlue")

In [ ]:
# Export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="BaseVIPR27km", snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("maxBlue", Out1, '', None, "0", "NONE", "NONE", "1_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

## Make a layer of all subregions that are relevant for 2023
this is a bit confusing because even though the Atlantic and Gulf marine were ran in zonation together, the Atlanic and Gulf extents were used to clip some indicators, so we need them pulled out seperate

### Gulf Marine

In [15]:
# bring in gulf marine raster and convert to vector
arcpy.conversion.RasterToPolygon(GOMm, "GOMmp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\GOMmp'>

In [16]:
# add field for subregion
arcpy.management.CalculateField("GOMmp", "SubRgn", '"Gulf of Mexico"', "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\GOMmp'>

### Things I tried that didn't work
I need to not have gaps between the base blueprint 2022 inland subregions and the new marine regions.
The base bluperint 2022 inland subreigon vector has smooth lines because it was made from HUCs and ecoregions, the gulf marine vector has non-simplified lines, so there were slivers when tried to merge them. So I rasterized the 2022 base blueprint inland subregions, then converted back to vector without simplifying the polygons. This worked with the gulf of marine vector, that had non simplified lines alogn the inland boundary.
I tried using Yvonne's atlantic marine raster, but it has a row of missing pixels in the north atlantic. So I tried using Yvonne's atlantic marine vector, but it has simplified polygons that leave gaps when merged with the now non simplified inland subregions. So I tried converting the vector to raster and got the same line of missing pixels. I was able to get it to convert to raster correctly after repairing the geomerty of the vector. Then I found I had problems with the overlap area of the gulf marine.
So I decided to use Yvonne's gulf marine vector with the non simplified inland polygons and then use the full marine extent to fill in the atlantic marine, but then I found that the southern boundary of the gulf marine vector had smooth lines, so when I added in the full marine there were slivers at the bottom. 


In [17]:
# make a copy of the gulf marine layer for edits
#with arcpy.EnvManager(outputCoordinateSystem=sr):
#    arcpy.management.CopyFeatures(GulfM, "GulfM1", '', None, None, None)

In [18]:
# make a copy of Yvonne's vector for edits
#with arcpy.EnvManager(outputCoordinateSystem=sr):
#    arcpy.management.CopyFeatures(AtlanticM, "AtlanticM1", '', None, None, None)

In [19]:
# add field to conver to raster with no simplify option
#arcpy.management.CalculateField("AtlanticM1", "SubRgn", '"Atlantic Marine"', "PYTHON3", '', "TEXT")

In [20]:
# do some cleaning to fix problem that is causing missing cells in raster
#arcpy.management.RepairGeometry("AtlanticM1", "DELETE_NULL", "ESRI")

In [21]:
# Convert to raster, have to do this so I won't get slivers when I merge with the base blueprint
#with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, extent = "BaseVIPR27km"):
#    arcpy.conversion.FeatureToRaster(in_features="AtlanticM1", field="SubRgn", out_raster="AtlanticM1r", cell_size=Rextent)

In [22]:
# convert Atlantic marine back to vector
#arcpy.conversion.RasterToPolygon("AtlanticM1r", "AtlanticM1rp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

### VIPR

In [23]:
# convert VIPR extent to vector
arcpy.conversion.RasterToPolygon(VIPR, "VIPRp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\VIPRp'>

In [24]:
# add field for subregion
arcpy.management.CalculateField("VIPRp", "SubRgn", '"Caribbean"', "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\VIPRp'>

### Rasterize then vectorize base blueprint 2022 subregions

In [25]:
# make a vector base blueprint 2022 subregion file that does not contain the marine subregions
arcpy.analysis.Select(baseSub, "baseSubNoM", "SubRgn NOT IN ('Marine Shelf and Extension', 'Marine Gulf Stream')")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\baseSubNoM'>

In [26]:
# convert 2022 base bluperint subregions to raster and snap to base blueprint 2022 extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, extent = "BaseVIPR27km"):
    arcpy.conversion.FeatureToRaster(in_features="baseSubNoM", field="SubRgn", out_raster="BaseSubRgn", cell_size=Rextent)

In [27]:
# convert 2022 base bluperint subregion raster back to vector
arcpy.conversion.RasterToPolygon("BaseSubRgn", "BaseSubRgnR", "NO_SIMPLIFY", "SubRgn", "SINGLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\BaseSubRgnR'>

In [28]:
arcpy.management.Dissolve("BaseSubRgnR", "BaseSubRgnRDis", "SubRgn", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\BaseSubRgnRDis'>

### Merge base blueprint 2022 inland, Gulf marine, and VIPR

In [29]:
# merge all subregions except South Atlantic, because that one has problems
arcpy.management.Merge("BaseSubRgnRDis;GOMmp;VIPRp", r"MergeSubRgnNoSA", 'SubRgn "SubRgn" true true false 27 Text 0 0,First,#,BaseSubRgnRDis,SubRgn,0,27,GOMmp,SubRgn,0,512,VIPRp,SubRgn,0,512', "NO_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MergeSubRgnNoSA'>

In [30]:
# Dissolve to clean up
arcpy.management.Dissolve("MergeSubRgnNoSA", "MergeSubRgnNoSADis", "SubRgn", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MergeSubRgnNoSADis'>

### Use Yvonne's full marine raster to add in South Atlantic marine
This was the only thing I was able to get to work without slivers, so long as I paired it with the raster gulf layer from Yvonne.

In [31]:
# use the full marine raster to add in the South Atlantic portion
# convert raster to vector
arcpy.conversion.RasterToPolygon(MarineRaster, "MarineRasterp", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MarineRasterp'>

In [32]:
# add field for subregion
arcpy.management.CalculateField("MarineRasterp", "SubRgn", '"Atlantic Marine"', "PYTHON3", '', "TEXT")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MarineRasterp'>

In [33]:
# erase the GOM from the above raster
arcpy.analysis.Erase("MarineRasterp", "MergeSubRgnNoSADis", "erase", None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\erase'>

### Merge previously created inland/Gulf/VIPR with the above South Atlantic Marine

In [34]:
# merge again, now including SA marine created from full marine
arcpy.management.Merge("MergeSubRgnNoSADis;erase", "MergeSubRgn", 'SubRgn "SubRgn" true true false 27 Text 0 0,First,#,MergeSubRgnNoSADis,SubRgn,0,27,erase,SubRgn,0,512', "NO_SOURCE_INFO")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MergeSubRgn'>

### do some testing for slivers and holes
I need to check this output to make sure that there are no slivers

In [35]:
# union with a big square to try to find holes
arcpy.analysis.Union([Square, "MergeSubRgn"], "GapCheck", "ALL", None, "NO_GAPS")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\GapCheck'>

In [40]:
# I didn't find any slivers in the GapCheck layer, so I'm ready to export this subregion vector
# make a copy of this just to change the name
arcpy.management.CopyFeatures("MergeSubRgn", Out2, '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\SEBlueprint2023SubregionsVIPR6'>

In [44]:
# export subregion vector layer to shapefile
with arcpy.EnvManager(outputCoordinateSystem=sr, extent = "BaseVIPR27km"):
    arcpy.conversion.FeatureClassToShapefile(Out2, Out2Folder)

### Make a layer showing the full extent of the SE Blueprint 2023

In [42]:
# add a field to vector subregion layer for raster converion
arcpy.management.CalculateField("MergeSubRgn", "raster", "1", "PYTHON3", '', "SHORT")

<Result 'D:\\SE_Blueprint_2023\\2_FullExtent\\FullExt.gdb\\MergeSubRgn'>

In [43]:
# convert new to raster 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, extent = "BaseVIPR27km"):
    arcpy.conversion.FeatureToRaster(in_features="MergeSubRgn", field="raster", out_raster=Out3, cell_size=Rextent)

In [ ]:
end = time.time()
print(end - start)